In [3]:
import torch
import gc
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformer_lens import HookedTransformer
from transformer_lens.loading_from_pretrained import get_pretrained_model_config, get_pretrained_state_dict, get_checkpoint_labels

In [4]:
torch.set_grad_enabled(False)

In [5]:
MODEL_NAME = "EleutherAI/pythia-6.9b"

In [10]:
HookedTransformer.from_pretrained?

Signature:
HookedTransformer.from_pretrained(
    model_name: str,
    fold_ln: Optional[bool] = True,
    center_writing_weights: Optional[bool] = True,
    center_unembed: Optional[bool] = True,
    refactor_factored_attn_matrices: Optional[bool] = False,
    checkpoint_index: Optional[int] = None,
    checkpoint_value: Optional[int] = None,
    hf_model: Optional[transformers.models.auto.modeling_auto.AutoModelForCausalLM] = None,
    device: Union[str, torch.device, NoneType] = None,
    n_devices: Optional[int] = 1,
    tokenizer: Optional[transformers.tokenization_utils_base.PreTrainedTokenizerBase] = None,
    move_to_device: Optional[bool] = True,
    fold_value_biases: Optional[bool] = True,
    default_prepend_bos: Optional[bool] = True,
    default_padding_side: Optional[Literal['left', 'right']] = 'right',
    dtype='float32',
    **from_pretrained_kwargs,
) -> 'HookedTransformer'
Docstring:
Load in a Pretrained Model.

Load in pretrained model weights to the HookedTransfor

In [6]:
model = HookedTransformer.from_pretrained(
    MODEL_NAME, 
    checkpoint_value=141000,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    dtype=torch.bfloat16,
)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-6.9b into HookedTransformer


In [8]:
model = HookedTransformer.from_pretrained(
    MODEL_NAME, 
    checkpoint_value=142000,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    dtype=torch.bfloat16,
)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-6.9b into HookedTransformer


In [14]:
cache_dir = "model_cache"
model = HookedTransformer.from_pretrained(
    MODEL_NAME, 
    checkpoint_value=130000,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    dtype=torch.bfloat16,
    **{"cache_dir": cache_dir},
)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-6.9b into HookedTransformer


In [7]:
model.generate("Mary and John went to the store. Afterwards, Mary gave some milk to", temperature=0.0)

  0%|          | 0/10 [00:00<?, ?it/s]

'Mary and John went to the store. Afterwards, Mary gave some milk to a neighbor’s child. The child drank the'

In [4]:
with torch.no_grad():
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, revision="step143000", torch_dtype=torch.float16)
    cfg = get_pretrained_model_config(MODEL_NAME, torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = HookedTransformer(cfg, tokenizer=tokenizer)
    state_dict = get_pretrained_state_dict(MODEL_NAME, cfg, hf_model, torch_dtype=torch.float16)
    model.load_state_dict(state_dict, strict=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
del hf_model
torch.cuda.empty_cache()  # Clear GPU cache
gc.collect()  # Run garbage collection

21

In [6]:
model.generate("Testing, testing, ")

  0%|          | 0/10 [00:00<?, ?it/s]

'Testing, testing, ercim.\n\nWe’ve just submitted'

In [11]:
del hf_model

In [13]:
gc.collect()

29

In [14]:
with torch.no_grad():
    hf_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, revision="step143000", torch_dtype=torch.float16)
    cfg = get_pretrained_model_config(MODEL_NAME, torch_dtype=torch.float16)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = HookedTransformer(cfg, tokenizer=tokenizer)
    state_dict = get_pretrained_state_dict(MODEL_NAME, cfg, hf_model, torch_dtype=torch.float16)
    model.load_state_dict(state_dict, strict=False)

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.91G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [15]:
import gc
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size())
    except:
        pass


<class 'torch.nn.parameter.Parameter'> torch.Size([12288, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([12288])
<class 'torch.nn.parameter.Parameter'> torch.Size([4096, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([16384, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([4096, 16384])
<class 'torch.nn.parameter.Parameter'> torch.Size([12288, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([12288])
<class 'torch.nn.parameter.Parameter'> torch.Size([4096, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([16384, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([4096, 16384])
<class 'torch.nn.parameter.Parameter'> torch.Size([50432, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([12288, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([12288])
<class 'torch.nn.parameter.Parameter'> torch.Size([4096, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([16384, 4096])
<class 'torch.nn.parameter.Parameter'> torch.S